## SIMULACION DE LOGS Y GRAFICO DE DATOS 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Generar timestamps para una semana-parte JOEL SIZA 

In [ ]:

fechas = pd.date_range(start='2023-01-01', periods=7)
timestamps = []
for fecha in fechas:
    for hora in range(24):
        for _ in range(3):  # 3 eventos por hora aprox
            timestamps.append(fecha + pd.Timedelta(hours=hora) + 
                             pd.Timedelta(minutes=np.random.randint(60)))

In [4]:
print(timestamps)

[Timestamp('2023-01-01 00:13:00'), Timestamp('2023-01-01 00:41:00'), Timestamp('2023-01-01 00:23:00'), Timestamp('2023-01-01 01:01:00'), Timestamp('2023-01-01 01:19:00'), Timestamp('2023-01-01 01:13:00'), Timestamp('2023-01-01 02:52:00'), Timestamp('2023-01-01 02:36:00'), Timestamp('2023-01-01 02:22:00'), Timestamp('2023-01-01 03:40:00'), Timestamp('2023-01-01 03:30:00'), Timestamp('2023-01-01 03:38:00'), Timestamp('2023-01-01 04:10:00'), Timestamp('2023-01-01 04:42:00'), Timestamp('2023-01-01 04:18:00'), Timestamp('2023-01-01 05:09:00'), Timestamp('2023-01-01 05:56:00'), Timestamp('2023-01-01 05:37:00'), Timestamp('2023-01-01 06:04:00'), Timestamp('2023-01-01 06:51:00'), Timestamp('2023-01-01 06:19:00'), Timestamp('2023-01-01 07:16:00'), Timestamp('2023-01-01 07:48:00'), Timestamp('2023-01-01 07:41:00'), Timestamp('2023-01-01 08:56:00'), Timestamp('2023-01-01 08:00:00'), Timestamp('2023-01-01 08:39:00'), Timestamp('2023-01-01 09:40:00'), Timestamp('2023-01-01 09:47:00'), Timestamp('20